## Data Understanding

### Data Loading

Import Library yang dibutuhkan dan load dataset yang akan digunakan.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


Kemudian mengimport data ratings dengan pandas lalu menampilkan untuk melihat struktur data dari ratings.

In [ ]:
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')
df_ratings

Dari output code di atas didapatkan informasi sebagai berikut.
- Data ratings terdiri dari 100836 baris dan 4 kolom.
- Terdapat 3 kolom fitur yaitu `userId`, `movieId`, dan `timestamp`
- Terdapat 1 kolom target yaitu `rating`


Selanjutnya, mengimport data movies dengan pandas lalu menampilkan untuk melihat struktur data dari movies.

In [ ]:
df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_movies

Dari output code di atas didapatkan informasi sebagai berikut.
- Data movies terdiri dari 9742 baris dan 3 kolom.
- Terdapat 3 kolom fitur yaitu `movieId`,`title`, dan `genres`

Lalu, mengimport data tags dengan pandas lalu menampilkan untuk melihat struktur data dari tags.

In [ ]:
df_tags = pd.read_csv('ml-latest-small/tags.csv')
df_tags

Dari output code di atas didapatkan informasi sebagai berikut.
- Data tags terdiri dari 3683 baris dan 4 kolom.
- Terdapat 4 kolom fitur yaitu `userId`, `movieId`, `tag`, dan `timestamp`

### Exploratory Data Analysis

#### Mengecek Tipe Data

Pada tahap ini, akan dilakukan pengecekan tipe data dari masing-masing kolom pada data ratings, movies, dan tags.

In [ ]:
df_movies.info()

Dari output code di atas tipe data dari semua kolom pada data movies sudah sesuai.

In [ ]:
df_ratings.info()

Dari output code di atas tipe data dari semua kolom pada data ratings sudah sesuai.

In [ ]:
df_tags.info()

Dari output code di atas tipe data dari semua kolom pada data tags sudah sesuai.

#### Mengecek Missing Value

Lalu akan dilakukan pengecekan missing value pada data ratings, movies, dan tags. Hal ini dilakukan agar dataset yang akan digunakan tidak mengandung missing value sehingga tidak menimbulkan error pada saat pemodelan.

In [ ]:
df_movies.isna().sum()

Dari output code di atas tidak terdapat missing value pada data movies.

In [ ]:
df_ratings.isna().sum()

Dari output code di atas tidak terdapat missing value pada data ratings.

In [ ]:
df_tags.isna().sum()

Dari output code di atas tidak terdapat missing value pada data tags.

#### Mengecek Duplikasi Data

Lalu akan dilakukan pengecekan duplikasi data pada data ratings, movies, dan tags. Hal ini dilakukan agar dataset yang digunakan tidak mengandung duplikasi data yang dapat mempengaruhi hasil analisis.

In [ ]:
df_movies.duplicated().sum()

Dari output code di atas tidak terdapat duplikasi data pada data movies.

In [ ]:
df_ratings.duplicated().sum()

Dari output code di atas tidak terdapat duplikasi data pada data ratings.

In [ ]:
df_tags.duplicated().sum()

Dari output code di atas tidak terdapat duplikasi data pada data ratings.

#### Mengecek Distribusi Data

Selanjutnya, akan dilakukan pengecekan distribusi data hanya pada data ratings karena hanya kolom ratings yang memiliki tipe data numerik dan yang akan digunakan.

In [ ]:
df_ratings.describe()

Dari output code di atas didapatkan informasi sebagai berikut.
- Rata-rata dari kolom ratings adalah 3.50
- Standar deviasi dari kolom ratings adalah 1.04
- Nilai minimum dari kolom ratings adalah 0.50
- Nilai 25% dari kolom ratings adalah 3.00
- Nilai 50% dari kolom ratings adalah 3.50
- Nilai 75% dari kolom ratings adalah 4.00
- Nilai maksimum dari kolom ratings adalah 5.00

Sehingga dapat disimpulkan bahwa distribusi data dari kolom ratings cenderung menumpuk di nilai 3.00 - 4.00. Hal ini bukanlah menjadi masalah karena data rating merupakan data yang bersifat subjektif dan dapat bervariasi.

Untuk memastikan distribusi data dari kolom ratings, akan dilakukan visualisasi distribusi data dari kolom ratings.

In [ ]:
df_ratings['rating'].hist()

Dari histogram di atas, grafik terlihat left-skewed yang menunjukkan bahwa mayoritas film memiliki rating di antara 3.0 - 4.0.

## Data Preparation

### Merge Data

Langkah selanjutnya adalah menggabungkan dataset sesuai dengan kebutuhan. Data ratings akan digabungkan dengan data movies dan data movies juga akan digabungkan dengan data tags.

#### Data Movies dan Data Tags

Penggabungan data movies dengan data tags dilakukan untuk mengetahui kesamaan antar film berdasarkan tag dan genres yang diberikan oleh user. Hal ini akan berguna untuk memberikan rekomendasi film yang memiliki kesamaan berdasarkan genres dan tag yang diberikan oleh user.

Langkah pertama adalah menggabungkan setiap tag yang diberikan oleh user berdasarkan `movieId`.

In [ ]:
df_tags = df_tags.groupby('movieId')['tag'].agg(lambda x: ' '.join(x)).reset_index()
df_tags.head()

Dari output code di atas dapat dilihat kolom tags sudah digabungkan berdasarkan `movieId`.

Langkah selanjutnya yaitu mengganti pemisah pada kolom genre menjadi sebuah whitespace.

In [ ]:
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.replace("|"," "))


Lalu akan digabungakan data movies dengan data tags berdasarkan `movieId`.

In [ ]:
df_movies_similarity = pd.merge(df_movies,df_tags,how='left',on='movieId')
df_movies_similarity.fillna(' ',inplace=True)
df_movies_similarity.head()

Dari output code di atas dapat dilihat kolom tags sudah digabungkan berdasarkan `movieId` meskipun terdapat beberapa missing value pada kolom tags. Missing value pada kolom tags ini akan diisi dengan whitespace.

Langkah Selanjutnya menggabungakan kolom tag dengan kolom genres menjadi satu kolom baru. Hal ini dilakukan agar nantinya dapat digunakan untuk menghitung kesamaan antar film berdasarkan tag dan genres hanya dengan satu kolom saja.

In [ ]:
df_movies_similarity['tags'] = df_movies_similarity['genres'] + " " + df_movies_similarity['tag']
df_movies_similarity.drop(columns=['genres','tag'],inplace=True)
df_movies_similarity.head()

#### Data Movies dan Data Ratings

Pada tahap ini, akan dilakukan penggabungan data ratings dengan data movies. Hal ini dilakukan agar nantinya dapat digunakan untuk memberikan rekomendasi film berdasarkan rating yang diberikan oleh user pada movie dengan mempertimbangkan genres.

In [ ]:
df_ratings_movie= pd.merge(df_ratings,df_movies,how='left',on='movieId')
df_ratings_movie.drop(columns=['timestamp'],axis=1,inplace=True)
df_ratings_movie['genres'] = df_ratings_movie['genres'].apply(lambda x: np.nan if x == '(no genres listed)' else x)
df_ratings_movie.dropna(inplace=True)
df_ratings_movie.reset_index()
df_ratings_movie.head()

Pada output code di atas, data movies dan data ratings sudah digabungkan berdasarkan `movieId`.

Langkah selanjutnya yaitu melakukan encoding dengan menjabarkan kolom genres menjadi beberapa kolom berdasarkan nilai yang ada pada kolom genres yang berisi nilai 1 atau 0.
Hal ini dilakukan karena model machine learning hanya dapat memproses data yang berupa numerik.

In [ ]:
df_ratings_movie['genres'] = df_ratings_movie['genres'].str.split(' ')
df_genres = df_ratings_movie['genres'].apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0).astype(int)
df_ratings_encode= pd.concat([df_ratings_movie, df_genres], axis=1)
df_ratings_encode.drop(columns=['genres'],inplace=True)
df_ratings_encode.head()

Dari output code di atas dapat dilihat kolom genres sudah dijabarkan dengan masing-masing jenis genres diwakili oleh satu kolom.

### Data Transformation

Setelah melakukan penggabungan data, langkah selanjutnya adalah melakukan transformasi data. Pada tahap ini, data yang berisi rating dan movie akan ditransformasi menjadi data yang berisi user, movie, dan rating. Sehingga data user dan movie akan menjadi X dan data rating akan menjadi y.

#### Data User

Pada tahap ini, akan dilakukan pembuatan data preferensi user berdasarkan rating yang diberikan oleh user pada setiap film dengan mempertimbangkan genres. Sehingga kolom yang tidak diperlukan akan dihapus.

In [ ]:
df_user = df_ratings_encode.copy()
df_user.drop(columns=['movieId','title'],inplace=True)
df_user

Selanjutnya setiap value 1 pada fitur genres akan dikalikan dengan rating yang diberikan oleh user pada film tersebut.

In [ ]:
for i in range(2, 21):
    genre_column = df_user.columns[i]
    df_user[genre_column] = df_user.apply(lambda row: row['rating'] if row[genre_column] == 1 else np.nan,axis=1)

df_user

Lalu akan dilakukan grouping berdasarkan `userId` dan dengan menghitung rata-rata dari fitur genres, sedangkan kolom rating akan dihapus karena sudah tidak diperlukan.

In [ ]:
genre_columns = df_user.columns[2:]
df_user_avg = df_user.groupby('userId')[genre_columns].mean().reset_index()
df_user_avg.fillna(0,inplace=True)
df_user_avg.head()

Dari output code di atas menunjukan bahwa data preferensi user sudah berhasil dibuat.

Langkah selanjutnya adalah melakukan penggabungan data preferensi user dengan data ratings berdasarkan `userId`. Hal ini bertujuan untuk menggantikan data user dengan data preferensi user.

In [ ]:
df_user = pd.merge(df_user,df_user_avg,how='left',on='userId')
df_user

Langkah selanjutnya melakukan drop data user yang sudah tidak diperlukan.

In [ ]:
df_user.drop(columns=df_user.columns[1:21],inplace=True)
df_user.head()

Lalu akan dilakukan rename fitur genres yang namanya berubah ketika melakukan penggabungan data.

In [ ]:
df_user.columns = ['userId'] + genre_columns.tolist()
df_user

#### Data Item

Selanjutnya, akan dilakukan pembuatan data movie dengan genres dengan menghapus kolom yang tidak berkaitan dengan movie.

In [ ]:
df_item = df_ratings_encode.copy()
df_item.drop(columns=['userId','rating','title'],inplace=True)
df_item.head()

Dari output code di atas, terlihat data movie (yang akan disebut item) yang akan disandingkan dengan data preferensi user (yang akan disebut user) hanya berisi fitur `movieId` dan fitur one hot encoding genres.

#### Data Rating

Selanjutnya, untuk data rating sebagai Y akan diambil kolom `rating` saja.

In [ ]:
rating = df_ratings_encode['rating'].values

### Scalling Data

Lalu data user dan data item akan di scalling dengan menggunakan Standard Scaler. Alasan Standard Scaler digunakan yaitu karena akan dilakukan pembagian rentang rating, jika value genre bernilai positif menandakan user menyukai film tersebut, sedangkan jika value genre bernilai negatif menandakan user tidak menyukai film tersebut. Atau jika pada data item, value genre bernilai positif menandakan film tersebut memiliki genre tersebut, sedangkan jika value genre bernilai negatif menandakan film tersebut tidak memiliki genre tersebut.

In [ ]:
scaler_user = StandardScaler()
scaler_item = StandardScaler()

scaler_user.fit(df_user[genre_columns])
scaler_item.fit(df_item[genre_columns])

df_user[genre_columns] = scaler_user.transform(df_user[genre_columns])
df_item[genre_columns] = scaler_item.transform(df_item[genre_columns])

In [ ]:
df_user.head()

Dari output code di atas jika value genre bernilai positif menandakan user menyukai genre tersebut, sedangkan jika value genre negatif menandakan user tidak menyukai genre tersebut, atau jika value bernilai 0 menandakan user tidak memiliki preferensi terhadap genre tersebut atau user tidak pernah menonton film dengan genre tersebut

In [ ]:
df_item.head()

Dari output code di atas jika value genre bernilai positif menandakan film tersebut memiliki genre tersebut, sedangkan jika value genre negatif menandakan film tersebut tidak memiliki genre tersebut, sedangkan tidak terdapat value 0 karena setiap film pasti memiliki genre.

Selanjutnya akan dilakukan scalling pada data rating dengan menggunakan MinMaxScaler. Hal ini bertujuan agar data rating memiliki rentang nilai antara 0 - 1.

In [ ]:
scaler = MinMaxScaler((-1,1))
rating = scaler.fit_transform(rating.reshape(-1,1))
rating

### Data Splitting

 Selanjutnya, data user, data item, dan data rating akan di split menjadi data train, data test, dan data validation. Hal ini bertujuan agar dapat dilakukan evaluasi model machine learning yang akan dibuat. Perbandingan data train, data test, dan data validation yang digunakan adalah 80% data train, 10% data test, dan 10% data validation. Hal ini dipertimbangkan berdasarkan jumlah data yang ada.

In [ ]:
user_train, user_test, item_train, item_test, rating_train, rating_test = train_test_split(df_user[genre_columns], df_item[genre_columns], rating.flatten(), test_size=0.2, random_state=42)
user_test, user_val, item_test, item_val, rating_test, rating_val = train_test_split(user_test, item_test, rating_test, test_size=0.5, random_state=42)

user_train.shape, user_test.shape, user_val.shape, item_train.shape, item_test.shape, item_val.shape, rating_train.shape, rating_test.shape, rating_val.shape


## Content-Based Filtering

Tahap selanjutnya yaitu membuat sistem rekomendasi film berdasarkan konten. Pada tahap ini akan dilakukan pembuatan model neural network dengan menggunakan data preferensi user dan data movie. Selain itu juga akan dibuat akan dihitung similarity antar film berdasarkan genres dan tag.

### Memory-Based Item Based

Pada perhitungan similarity antar film berdasarkan genres dan tag, akan digunakan cosine similarity. Hal ini dilakukan karena cosine similarity dapat mengukur kesamaan antar film berdasarkan genres dan tag. Tetapi sebelum itu, akan dilakukan ekstraksi data tags dengan menggunakan CountVectorizer. Dengan menggunakan CountVectorizer, akan dihitung frekuensi kemunculan setiap kata pada kolom tags.

In [ ]:
cv = CountVectorizer(max_features = 5000, lowercase=True)
vectors = cv.fit_transform(df_movies_similarity['tags']).toarray()

Selanjutnya akan dilakukan perhitungan similarity antar vector features tags dengan menggunakan cosine similarity.

In [ ]:

cosine_sim = cosine_similarity(vectors)

Setelah itu akan dilakukan visualisasi similarity antar film berdasarkan tags.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cosine_sim)
plt.title('Cosine Similarity Heatmap')
plt.show()

Dari diagram heatmap di atas, terlihat bahwa banyak daerah yang memiliki warna yang cendrung terang yang menunjukkan bahwa terdapat banyak kesamaan antar film berdasarkan tags.

Selanjutnya akan dibentuk suatu function yang berfungsi mengambil 10 movie yang memiliki similarity tertinggi berdasarkan tags.

In [ ]:
def get_recommendation_movie(title):
    index = df_movies_similarity[df_movies_similarity['title'] == title].index[0]
    similarity_score = cosine_sim[index] 
    similarity_place = sorted(enumerate(similarity_score),key=lambda x: x[1],reverse=True)[1:11]
    similarity_list = []    
    for i in similarity_place:
        similarity_list.append([df_movies_similarity.iloc[i[0], 1]] + [i[1]])
    return similarity_list

Dari function di atas, inputan dari function tersebut yaitu judul dari film yang ingin dicari movie yang memiliki similarity tertinggi berdasarkan tags. Nantinya function tersebut akan mengambil similarity movie tersebut dengan movie lainnya berdasarkan indeks. Selanjutnya akan dilakukan sorting, lalu mengambil top 10 movie yang memiliki similarity tertinggi.

In [ ]:
top_10_recommendation = get_recommendation_movie('Toy Story (1995)')
top_10_recommendation

Dari output code di atas, terlihat top 10 movie yang memiliki similarity tertinggi dengan movie Toy Story (1995).

### Model-Based Deep Learning

Tahap selanjutnya yaitu membuat model neural network dengan menggunakan data preferensi user dan data movie. Model neural network yang akan dibuat akan menggunakan data preferensi user dan data movie sebagai X dan data rating sebagai y.

In [ ]:
features = 19

user_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='linear')
])

item_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='linear')
])

input_user = tf.keras.Input(shape=(features,))
vector_user = user_model(input_user)
vector_user = tf.linalg.l2_normalize(vector_user, axis=1)

input_item = tf.keras.Input(shape=(features,))
vector_item = item_model(input_item)
vector_item = tf.linalg.l2_normalize(vector_item, axis=1)

output = tf.keras.layers.Dot(axes=1)([vector_user, vector_item])

model = tf.keras.Model(inputs=[input_user, input_item], outputs=output)

model.summary()

Pada code di atas, secara garis besar terdapat dua model sequential yang akan digunakan. Model sequential pertama yaitu model untuk data user dan model sequential kedua yaitu model untuk data item. Nantinya output dari kedua model tersebut akan digabungkan dengan menggunakan dot product.

Selanjutnya akan dilakukan compile model dengan menggunakan optimizer Adagrad, loss function mean squared error, dan metrics mean squared error.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.01), loss='mean_squared_error', metrics=['mse'])

Model akan difit dengan menggunakan 100 epochs dan batch size 256. Ini berarti model akan melakukan iterasi sebanyak 100 kali dengan menggunakan 256 data pada setiap iterasinya.

In [ ]:
history = model.fit([user_train, item_train], rating_train, epochs=100, verbose=2, batch_size=256, validation_data=([user_val, item_val], rating_val))

Selanjutnya akan dilakukan prediksi top 10 movie berdasarkan suatu preferensi user. Berikut merupakan contoh preferensi user yang akan digunakan.

In [ ]:
new_user_id = 1000
new_adventure = 4
new_animation = 5
new_children = 4
new_comedy = 3
new_fantasy = 4
new_romance = 1
new_action = 3
new_crime = 1
new_thriller = 1
new_mystery = 1
new_horror = 1
new_drama = 1
new_war = 1
new_western = 1
new_scifi = 1
new_musical = 1
new_filmnoir = 1
new_imax = 1
new_documentary = 1

new_user = pd.DataFrame([[new_user_id, new_adventure, new_animation, new_children, new_comedy, new_fantasy, new_romance, new_action, new_crime, new_thriller, new_mystery, new_horror, new_drama, new_war, new_western, new_scifi, new_musical, new_filmnoir, new_imax, new_documentary]], columns=['userId', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Action', 'Crime', 'Thriller', 'Mystery', 'Horror', 'Drama', 'War', 'Western', 'Sci-Fi', 'Musical', 'Film-Noir', 'IMAX', 'Documentary'])
new_user

Dari output code tersebut terlihat user 1000 memiliki preferensi terhadap film dengan genres Adventure, Animation, Childern, Fantasy, dan Action.

Sebelum melakukan prediction, akan dilakukan scaling pada data preferensi user 1000 dan juga akan dilakukan penyesuaian dimensi agar dapat digunakan pada model yang telah dibuat.

In [ ]:
new_user[genre_columns] = scaler_user.transform(new_user[genre_columns])

new_user = np.tile(new_user[genre_columns], (df_item.shape[0], 1))
new_user.shape

Setelah melakukan penyesuaian dimensi maka dimensi dari data user 1000 sama dengan dimensi dari data item yaitu (100789, 19).

Selanjutnya akan dilakukan prediction dengan data user 1000.

In [ ]:
predictions = model.predict([new_user, df_item[genre_columns]])


In [ ]:
predictions = scaler.inverse_transform(predictions)
sorted_predictions = np.argsort(predictions, axis=0)[::-1].flatten()
sorted_item = df_ratings.index.to_numpy()[sorted_predictions].flatten()
sorted_item

Setelah dilakukan prediksi, output dari prediksi tersebut akan dilakukan inverse transform dan melakukan sorting untuk mendapatkan top 10 movie yang direkomendasikan.

In [ ]:
dic_predictions = {
    'userId': np.full((df_item.shape[0],), new_user_id),
    'index': df_ratings_movie.iloc[sorted_item].index,
    'predictions': predictions[sorted_predictions].flatten()
} 
df_predictions = pd.DataFrame(dic_predictions)
df_predictions.set_index('index', inplace=True)
df_predictions = pd.merge(df_predictions, df_ratings_movie, how='left', left_index=True, right_index=True).reset_index(drop=True)
df_predictions.drop_duplicates(subset=['movieId'], inplace=True)
df_predictions.drop(columns=['userId_y', 'rating'], inplace=True)
df_predictions.rename(columns={'userId_x': 'userId'}, inplace=True)
df_predictions.reset_index(drop=True,inplace=True)
df_predictions.head(10)

Dari ouput code di atas, terlihat top 10 movie yang direkomendasikan untuk user 1000 berdasarkan preferensi user tersebut.

## Evaluation

Langkah terakhir yaitu melakukan evaluasi model yang telah dibuat.

Evaluation pertama yaitu melakukan evaluasi model dengan menggunakan data validation. Hal ini bertujuan untuk mengetahui seberapa baik model yang telah dibuat dengan menggunakan data yang belum pernah dilihat sebelumnya.

In [ ]:
val_mse, val_loss = model.evaluate([user_val, item_val], rating_val, verbose=2)

Selanjutnya MSE dari train, test, dan validation akan divisualisasikan agar dapat dilihat perbandingannya.

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(history.history['mse'], label='Training MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.plot(val_mse, label='Validation mse', marker='o', markersize=10)
plt.legend()
plt.title('MSE Graph')
plt.show()

Berdasarkan grafik di atas, terlihat bahwa MSE dari data train, data test, dan data validation cenderung stabil dan tidak terlalu berbeda. Hal ini menunjukkan bahwa model yang telah dibuat tidak overfitting dan tidak underfitting.